In [ ]:
import matplotlib
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from scipy.spatial.transform import Rotation

dwxdw1 = np.array([
    [0, 0, 0],
    [0, 0, -1],
    [0, 1, 0]
], dtype=np.float64)

dwxdw2 = np.array([
    [0, 0, 1],
    [0, 0, 0],
    [-1, 0, 0]
], dtype=np.float64)

def exp(w1, w2, w3):
    theta = np.linalg.norm([w1, w2, w3])
    if theta < 1e-8:
        return np.eye(3)
    w1 /= theta
    w2 /= theta
    w3 /= theta
    wx = np.array([
        [0, -w3, w2],
        [w3, 0, -w1],
        [-w2, w1, 0]
    ])
    return np.eye(3) + wx * np.sin(theta) + (wx @ wx) * (1 - np.cos(theta))

def compute_residuals_jacobian(zi, ti, s, w1, w2, d, w3):
    R = exp(w1, w2, w3)
    ri = zi - (s * (R @ ti)[2] + d)
    drids = -(R @ ti)[2]
    dridw1 = -s * (dwxdw1 @ R @ ti)[2]
    dridw2 = -s * (dwxdw2 @ R @ ti)[2]
    Ji = np.array([drids, dridw1, dridw2, -1])
    return ri, Ji

def optimize(z, t, parameters, w3, n_iter=10):
    print('---------------------------------------------')
    print('Starting optimization')
    print('---------------------------------------------')
    for iteration in range(n_iter):
        r = []
        J = []
        for i in range(len(z)):
            ri, Ji = compute_residuals_jacobian(z[i], t[i], *parameters, w3)
            r.append(ri)
            J.append(Ji)
        r = np.stack(r)
        J = np.stack(J)
        parameters -= np.linalg.inv(J.T @ J) @ J.T @ r
        # new_t = ((parameters[0] * exp(*parameters[1:3], 0)) @ t.T + parameters[3]).T
        print(f'cost: {np.square(r).sum()}')
    print(f'Solution: {parameters}')
    print(f'Retrieved scale: {1 / parameters[0]}')
    return parameters

In [ ]:
old_t = np.random.rand(40, 3)
z = old_t[:, 2].copy() + np.random.randn(old_t.shape[0]) / 20

R_gt = Rotation.random().as_matrix()
t_gt = np.random.rand(3, 1)
s_gt = 0.78
t = (s_gt * R_gt @ old_t.T + t_gt).T

cmap = matplotlib.cm.get_cmap('jet')
colors = cmap(np.linspace(0, 1, z.size))

In [ ]:
initializations = [np.array([1, 0, 0, 0], dtype=np.float64)]
for angle_amplitude in np.linspace(0, np.pi, 4)[1:]:
    for angle_direction in ([1, 0], [0, 1], [np.sqrt(0.5), np.sqrt(0.5)]):
        initializations.append(np.array([1, *np.array(angle_direction) * angle_amplitude, 0]))

for initialization in initializations:
    beta = optimize(z, t, initialization, 0, n_iter=100)
    if beta[0] > 0:
        break
    else:
        print('---------------------------------------------')
        print('Converged to an asymmetric solution (s < 0),\nrestarting optimization with different initialization...')

In [ ]:
beta = np.array([1, 0, 0, 0], dtype=np.float64)

for iteration in range(20):
    r = []
    J = []
    for i in range(len(z)):
        r.append(compute_residuals(z[i], t[i], *beta))
        J.append(compute_jacobian(z[i], t[i], *beta[:-1]))
    r = np.stack(r)
    J = np.stack(J)
    beta -= np.linalg.inv(J.T @ J) @ J.T @ r
    new_t = ((beta[0] * exp(*beta[1:3], 0)) @ t.T + beta[3]).T
    print(f'cost: {np.square(r).sum()}')

In [ ]:
1 / beta[0]

In [ ]:
%matplotlib qt
fig = plt.figure(figsize=(12, 12))
ax = fig.add_subplot(111, projection='3d')
real_alt = new_t.copy()
real_alt[:, 2] = z
ax.scatter(*real_alt.T, c=colors)
ax.scatter(*new_t.T, c=colors * 0.7)